<div style="padding-bottom:50px">
 <img src="https://res.cloudinary.com/djz27k5hg/image/upload/v1657885120/logos/univr_logo_rspn8o.jpg"  width=250 align='left' style="margin-top:30px"/>
<img src="https://res.cloudinary.com/djz27k5hg/image/upload/v1637335206/logos/Logo_des_Forschungszentrums_J_C3_BClich_seit_2018_hcliq4.svg"  width=250 align='left' style="margin-top:30px; margin-left:50px"/>
<img src="https://res.cloudinary.com/djz27k5hg/image/upload/v1637657234/logos/HBP_horizontal_logo_qtcyzn.png" width="300" align='left' style="margin-left:50px">
</div> 

<br><br><br><br>

# GOMoDo
### GPCR Online Modeling and Docking
<br><br>


This notebook was desinged to be run in Google colab. If you'd like to open this notebook in colab, you can use the following link.

<div style='padding:15px'>
<a href="https://colab.research.google.com/github/rribeiro-sci/pygomodo/blob/main/Modelling_GOOGLE_COLAB.ipynb" target="_blank">
<img alt="Colab" src="https://res.cloudinary.com/djz27k5hg/image/upload/v1637335713/badges/colab-badge_hh0uyl.svg" height="25" style="margin:20px">
</a>

</div>  

## Setup

To run the GOMoDo within Google Colab, you'll need to run the following installation commands. 

## Colab Tips

Get Notified of completed cell executions 🔔

Colab can notify you of completed executions even if you switch to another tab, window, or application. You can enable it via Tools → Settings → Site → Show desktop notifications (and allow browser notifications once prompted) to check it out.

<img src="https://miro.medium.com/max/1400/1*i5apguaDKcITqss1h_ZA3g.jpeg" width="480" height="320">

## Install Dependencies

In [ ]:
#@title Set Modeller key
MODELLER_KEY = 'MODELIRANJE' #@param {type:"string"}


In [ ]:
#@title Install Modeller 
#Download Modeller
!wget https://salilab.org/modeller/10.2/modeller_10.2-1_amd64.deb

#Install Modeller
!dpkg -i modeller_10.2-1_amd64.deb 

#read input file
fin = open("/usr/lib/modeller10.2/modlib/modeller/config.py", "rt")
#read file contents to string
data = fin.read()
#replace all occurrences of the required string
data = data.replace('xxx', MODELLER_KEY)
#close the input file
fin.close()
#open the input file in write mode
fin = open("/usr/lib/modeller10.2/modlib/modeller/config.py", "wt")
#overrite the input file with the resulting data
fin.write(data)
#close the file
fin.close()

!cp -r /usr/lib/modeller10.2/modlib/modeller /usr/local/lib/python3.7/dist-packages/
!ln -s /usr/lib/modeller10.2/lib/x86_64-intel8/python3.3/_modeller.so* /usr/local/lib/python3.7/dist-packages/_modeller.so



In [ ]:
#@title Install HH-suite
## Install HH-SUITE
import os
import multiprocessing as mp
cpu = mp.cpu_count()
homedir=os.getcwd()

!git clone https://github.com/soedinglab/hh-suite.git
!mkdir -p hh-suite/build
os.chdir('hh-suite/build')
!cmake -DCMAKE_INSTALL_PREFIX=. ..
!make -j $cpu
!make install

os.chdir(homedir)
os.environ['HHSUITEPATH']=os.path.join(homedir, 'hh-suite/build/bin/' )
os.environ['HHSUITESCRIPTS']=os.path.join(homedir, 'hh-suite/scripts/' )
##Install PDBX
!git clone https://github.com/soedinglab/pdbx.git
os.chdir('pdbx')
!mkdir build
os.chdir('build')
!cmake -DUserInstallOption=ON ../
!make install
os.chdir(homedir)


In [ ]:
#@title Install pyGOMoDo

# download SSB source code and install dependencies
!git clone https://github.com/rribeiro-sci/pygomodo.git --quiet
!pip install -r pygomodo/requirements.txt

## unzip databases
os.chdir('pygomodo/src/databases')
!tar -xvf ProcessedPdbs_wo_isoforms.tar.xz
!tar -xvf pyGOMODO_db.tar.xz
!tar -xvf costum_db.tar.xz

!chmod -R 777 ProcessedPdbs_wo_isoforms/
!chmod -R 777 pyGOMODO_db/
!chmod -R 777 costum_db/
os.chdir(homedir)

#test bioservices
from bioservices import UniProt
u = UniProt(verbose=False)

#Set Interatictive tabe (for Google COLAB)
%load_ext google.colab.data_table

#Import pygomodo
from pygomodo.src.gomodo import *

## Modelling with pyGOMoDo

In [ ]:
#@markdown ##### Insert UniProt ID or Fasta:
uniprotid = "P21918" #@param {type:"string"}
fasta = "" #@param {type:"string"}

#@markdown ##### Insert Job Name:
jobname = "dopamine5"#@param {type:"string"}

#Initialize GOMODO instance giving a jobname (this will be the name of the results directory)
m = gomodo(jobname=jobname)

In [ ]:
#@title Create HHM profile

#@markdown HHBlits parameters:
#Loop = "No" #@param ["Yes", "No"]
rounds = 2 #@param [1,2,3,4] {type:"raw"}

HHMprofile = m.createHHMprofile(uniprotID=uniprotid, unisequence=fasta, ncpus=cpu, rounds=rounds)
HHMprofile

In [ ]:
#@title Search for homologous structures

Database = "Default" #@param ["Default"]

structures = m.searchTemplates(ncpus=cpu)
structures

In [ ]:
#@title
#Select the templates by changing the numbers inside the square brackets
templates=[structures.iloc[91].Hit,structures.iloc[195].Hit] 
templates

In [ ]:
#@title Create models

#@markdown Modeller options 

number_of_models =  4#@param  {type:"raw"}
loop_refinement = "No" #@param ["Yes", "No"]
nloops = 0#@param {type:"raw"}
trim = 'No' #@param ['Yes', 'No']


#Start modelling
models = m.makeModels(templates=templates, nmodels=number_of_models, ncpus=cpu, loop=loop_refinement,nloop=nloops, trim=trim)
models

In [ ]:
#@title 3D visualization of the Models
#vizualize model in 3D
m.vizModels()

## QMEANBRANE

The swissmodel API requires an email. Therefore a valid email must be given.  This colab does not collect, store, use or distribute any kind of personal data!

In [ ]:
m.qmeanbrane(models=['Model.B99990001.pdb','Model.B99990002.pdb'], email='rui.ribeiro@univr.it') ## change list of models and XX accordingly. 

## Save Files to Google Drive

In [ ]:
from google.colab import drive
drive.mount('/gdrive')

In [ ]:
m.gdrive(path='/gdrive/MyDrive/') ## change XX accordingly. 